In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.potentials import patchyProteinMarkovSwitch
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS

In [ ]:
# Define CTMSMs
# MSM for particle type0
MSMtype = 0
seed = 0 # If seed = -1 uses random device as seed
markovModel0 = ctmsm(MSMtype, seed) # If no rate matrix specified, yields ratematrix [[0]]
D0list = np.array([1.0])
Drot0list = np.array([1.0])
markovModel0.setD(D0list)
markovModel0.setDrot(Drot0list)

# MSM for particle type1 
MSMtype = 1
ratematrix = np.array([[-0.5,0.5],[6.0,-6.0]])
seed = 0 # If seed = -1 uses random device as seed
markovModel1 = ctmsm(MSMtype, ratematrix, seed)
D1list = np.array([0.5, 1.0])
Drot1list = np.array([0.5, 1.0])
markovModel1.setD(D1list)
markovModel1.setDrot(Drot1list)

# Create MSMlist
MSMlist = [markovModel0, markovModel1] 

In [ ]:
# Particle definition
numparticles = 2
p1type = 0
p2type = 1
p1state = 0
p2state = 1
positions = [np.array([1.,0,0]), np.array([-1.,0.,0.])]
orientations = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]
part1 = msmrd2.particle(p1type, p1state, D0list[p1state], Drot0list[p1state], positions[0], orientations[0])
part2 = msmrd2.particle(p2type, p2state, D1list[p2state], Drot1list[p2state], positions[1], orientations[1])
partlist = msmrd2.integrators.particleList([part1, part2])

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 0 # If seed = -1 uses random device as seed
bodytype = 'rigidbody' # Mix indicates there can be different types of rigidbody particles
integrator = odLangevinMS(MSMlist, dt, seed, bodytype) 
integrator.setKbT(1.0)

In [ ]:
# Define boundary (choose either spherical or box)
radius = 2.0 #3.0
sphereBoundary = msmrd2.sphere(radius,'reflective')
integrator.setBoundary(sphereBoundary)

In [ ]:
# Define Patchy Protein potential
sigma = 1.0
strength = 65
# This values are fixed and should match those used to determine metastable states in potential and trajectory.
patchesCoordinates1 = [np.array([1.,0.,0.]), 
                       np.array([0.,1.,0.]),
                       np.array([0.,0.,1.]),
                       np.array([-1.,0.,0.]),
                       np.array([0.,-1.,0.]),
                       np.array([0.,0.,-1.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
# Defines patchy protein potential
potentialPatchyProteinMS = patchyProteinMarkovSwitch(sigma, strength, patchesCoordinates1, patchesCoordinates2)
# Incorporate potential into integrator
integrator.setPairPotential(potentialPatchyProteinMS)

In [ ]:
# Integrate particle list and print only positions. Adds an extra far away particle to 
# emulate change of color/conformation since VMD can only handle a fixed number of particle with fixed types.
timesteps = 500000 #50000000
stride = 100 #10 #1000
datafile  = open('../../data/vmd/patchyProteinsMS.xyz', 'w')
farAwayPostion = np.array([30*radius, 0, 0])
numSpheres2Draw= 15
# Define reference vectors for plotting small protein
refVec1 = np.array([0., 1., 0.])
refVec2 = np.array([-1., 0., 0.])
refVec3 = np.array([0., -1., 0.])
refVec4 = np.array([0., 0., 1.])
refVec5 = np.array([0., 0., -1.])
# Loop over all timesteps, read data and output plot.
for i in  range(timesteps):
    if i%stride == 0:
        datafile.write(str(numSpheres2Draw) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[2], part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[3], part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[4], part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[5], part.orientation)
                # Main particle 1
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                # Pacthes
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v6)) + '\n')
            if part.type == 1:
                    v0 = part.position
                    #if part.state == 0:
                    v1 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                    v2 = v0 + 0.3*sigma*quats.rotateVec(refVec1, part.orientation)
                    v3 = v0 + 0.3*sigma*quats.rotateVec(refVec2, part.orientation)
                    v4 = v0 + 0.3*sigma*quats.rotateVec(refVec3, part.orientation)
                    v5 = v0 + 0.3*sigma*quats.rotateVec(refVec4, part.orientation)
                    v6 = v0 + 0.3*sigma*quats.rotateVec(refVec5, part.orientation)
                    # Main particle 2
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v2)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v3)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v4)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v5)) + '\n')
                    datafile.write('type_3' + ' ' + ' '.join(map(str, v6)) + '\n')
                    if part.state == 1:
                        datafile.write('type_4' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')
                    else:
                        datafile.write('type_4' + ' ' + ' '.join(map(str, v1)) + '\n')
                    
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteinsMSV2(numparticles = numparticles, tclfname = "../../data/vmd/patchyProteinsMS2vmd.tcl")
print("Percentage complete: ", 100, " %")

In [ ]:
msmrdvis.generateTCL_patchyProteinsMSV2(numparticles = numparticles, tclfname = "../../data/vmd/patchyProteinsMS2vmd.tcl")

In [ ]:
## Old version of plotting patchy proteinMS
# v0 = part.position
# if part.state == 0:
#     v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
#     # Main particle 2
#     datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
#     datafile.write('type_3' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')
#     # Patch
#     datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
#     datafile.write('type_4' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')

# else:
#     v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
#     # Main particle 2
#     datafile.write('type_0' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')
#     datafile.write('type_3' + ' ' + ' '.join(map(str, v0)) + '\n')
#     # Patch
#     datafile.write('type_2' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')
#     datafile.write('type_4' + ' ' + ' '.join(map(str, v1)) + '\n')